# Data Collection

## Articles

- [Oklahoma schools rank 50th in the nation in latest education quality study](https://www.oklahoman.com/story/news/education/2025/07/24/oklahoma-schools-ranked-nearly-the-worst-in-the-nation-in-new-study/85310196007/)
- [Public School Rankings by State 2025](https://worldpopulationreview.com/state-rankings/public-school-rankings-by-state)
- [Worst School Districts by State 2025](https://worldpopulationreview.com/state-rankings/worst-school-districts-by-state)
- [2025 Best School Districts in Massachusetts](https://www.niche.com/k12/search/best-school-districts/s/massachusetts/)
- [2025 Best School Districts in California](https://www.niche.com/k12/search/best-school-districts/s/california/)

### Palo Alto, CA

- [Henry M. Gunn High School](https://gunn.pausd.org/)
  - [Website](https://gunn.pausd.org/)
  - [Niche.com](https://www.niche.com/k12/henry-m-gunn-high-school-palo-alto-ca/)
  - [Yelp.com](https://www.yelp.com/biz/henry-m-gunn-high-school-palo-alto-2?osq=henry+m+gunn+high+school)
- [Palo Alto Highschool](https://www.paly.net/)
  - [Website](https://www.paly.net/)
  - [Niche.com](https://www.niche.com/k12/palo-alto-high-school-palo-alto-ca/)
  - [Yelp.com](https://www.yelp.com/biz/palo-alto-high-school-palo-alto?osq=palo+alto+high+school)
- [Hope Technology School - Private](https://hopetechschool.org/)
  - [Website](https://hopetechschool.org/)
  - [Yelp.com](https://www.yelp.com/biz/hope-technology-school-palo-alto-6)
- [Palo Alto Middle College High School](https://mc.pausd.org/)
  - [Website](https://mc.pausd.org/)
  - [Yelp.com](https://www.yelp.com/biz/palo-alto-middle-college-high-school-los-altos-hills?osq=middle+college+high+school)

### Oklahoma City, OK

- [Boulevard Academy](https://boulevardacademy.edmondschools.net/o/boulevardacademy)
  - [Website](https://boulevardacademy.edmondschools.net/o/boulevardacademy)
- [Memorial High School](https://memorial.edmondschools.net/o/memorial)
  - [Website](https://memorial.edmondschools.net/o/memorial)
  - [Niche.com](https://www.niche.com/k12/memorial-high-school-edmond-ok/)
  - [Yelp.com](https://www.yelp.com/biz/memorial-high-school-tulsa?osq=memorial+high+school)
- [North High School](https://north.edmondschools.net/o/north)
- [Santa Fe High School](https://santafe.edmondschools.net/o/santafe)


In [7]:
import pandas as pd
import praw

## Connect to Reddit API

In [8]:
# default credentials stored in `praw.ini` file
reddit = praw.Reddit("default")

subreddit = reddit.subreddit("education")

for post in subreddit.hot(limit=5):
    print(post.title, post.score, post.id, post.url)

Welcome to r/Education! Please read before posting! 141 b5bbjr https://www.reddit.com/r/education/comments/b5bbjr/welcome_to_reducation_please_read_before_posting/
In 2024, average US based teacher made about 73 cents for every dollar earned by a college-educated peer 188 1nrfw6x https://www.reddit.com/r/education/comments/1nrfw6x/in_2024_average_us_based_teacher_made_about_73/
In news that should shock literally no one in this sub, a watchdog group has found that administrator pay and student performance are inversely correlated. 154 1nrcd72 https://www.reddit.com/r/education/comments/1nrcd72/in_news_that_should_shock_literally_no_one_in/
Students just don't care anymore about actually learning to write 65 1nr6z12 https://www.reddit.com/r/education/comments/1nr6z12/students_just_dont_care_anymore_about_actually/
Built a U.S. State Capitals learning app & would love feedback from educators 1 1nruquz https://www.reddit.com/r/education/comments/1nruquz/built_a_us_state_capitals_learning_

In [9]:
query = "Palo Alto School District High School"

for submission in reddit.subreddit("all").search(query, limit=10):
    print(f"[ALL] Title: {submission.title}")
    print(f"Subreddit: {submission.subreddit}")
    print(f"URL: {submission.url}")
    print("-" * 80)

    # post content
    print(f"Post text: {submission.selftext[:500]}")  # first 500 char of body
    print("-" * 80)

    # grab top level comments
    submission.comments.replace_more(limit=0)  # flatten MoreComments
    for comment in submission.comments[:3]:
        print(f"Comment: {comment.body[:200]}")
    print("-" * 80)

[ALL] Title: Bay Area parents — when did you start planning high school choices tied to school districts and housing?
Subreddit: bayarea
URL: https://www.reddit.com/r/bayarea/comments/1nkg269/bay_area_parents_when_did_you_start_planning_high/
--------------------------------------------------------------------------------
Post text: Hi everyone,

My kid just started middle school in San Jose, and I’m already realizing how quickly high school decisions come up. It feels overwhelming since in the Bay Area, school choices are often tied so closely to housing and community.

For those who’ve gone through this, when did you start planning for high school? Did you focus more on academics, extracurriculars, or the overall reputation of the district? And did housing location play a big role in your decision?

I came across this bre
--------------------------------------------------------------------------------
Comment: Your post has been removed because your account doesn't have enough karma.

In [11]:
var_subreddit = "paloalto"
query = "high school"

subreddit = reddit.subreddit(var_subreddit)

for submission in subreddit.search(query, limit=10):
    print(f"Title: {submission.title}")
    print(f"Score: {submission.score}")
    print(f"Author: {submission.author}")
    print(f"URL: {submission.url}")
    print("-" * 40)

Title: Palo Alto high school (or Paly)
Score: 15
Author: ButterscotchSorry423
URL: https://www.reddit.com/r/paloalto/comments/1lufa8d/palo_alto_high_school_or_paly/
----------------------------------------
Title: supporting the family of Ash who our Palo Alto High School community tragically lost last week
Score: 204
Author: Longjumping_Net3070
URL: https://www.reddit.com/r/paloalto/comments/1jaobez/supporting_the_family_of_ash_who_our_palo_alto/
----------------------------------------
Title: What’s happening with parking along El Camino Real between Stanford and Palo Alto High School? 
Score: 13
Author: Bear650
URL: https://i.redd.it/5okftod42pde1.jpeg
----------------------------------------
Title: How can my kids attend Palo Alto high school
Score: 6
Author: Ok_Ingenuity6400
URL: https://www.reddit.com/r/paloalto/comments/16vepkb/how_can_my_kids_attend_palo_alto_high_school/
----------------------------------------
Title: Stanford undergrad willing to tutor Math to high school stud